# Transfer learning - Fine-tuning



## Fine Tuning

In [0]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [0]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
testset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [0]:
for images, labels in trainloader:
  print(images.size(), labels.size())
  break

In [0]:
model = models.vgg16(pretrained=True)
model

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [0]:
model.classifier

In [0]:
for i in range(0,7):
  model.classifier[i].requires_grad = True

In [0]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096,512),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1)
                      )


In [0]:
model.classifier

In [0]:
model.features

In [0]:
criterion = nn.NLLLoss()

In [0]:
from torch.optim import Adam

model = model.to(device)
optimizer = Adam(model.parameters())


## Training from the Fully Connected Network onwards

### Re-training the model

In [0]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

### The accuracy of the model

In [0]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

## Un-freezing & training on the LAST CNN block onwards

### Re-training the model

In [0]:
for i in range(24,31):
  model.features[i].requires_grad = True


In [21]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.37638935446739197
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.4667692184448242
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.4867028594017029
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.7602155208587646
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.46336594223976135
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.48384594917297363
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.6412556171417236
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.3660600483417511
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.6906344294548035
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.6811375021934509
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.30804815888404846
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.35521185398101807
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.6848178505897522
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.5894997715950012
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.3391475677490234

### The accuracy of the model

In [22]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 83.4375% 


## Un-freezing & training on the LAST TWO CNN block onwards

### Re-training the model

In [0]:
for i in range(17,24):
  model.features[i].requires_grad = True

In [24]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.4675794839859009
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.25580158829689026
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.5276210904121399
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.3741523027420044
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.5076795220375061
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.3216595947742462
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.3571995198726654
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.2978670001029968
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.36628425121307373
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.45307356119155884
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.4280996024608612
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.3617429733276367
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.34377187490463257
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.4511021375656128
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.3250049352645874


### The accuracy of the model

In [25]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 82.1875% 
